In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../230817_intent_labeled_by_6.csv')

In [3]:
# df.drop(columns={'label','intent'})
# self.labels = {
#                 0:"주문", 1:"배송", 2:"매장", 3:"AS", 
# 4:"제품_정보", 5:"제품_재고"
#             }
df

,query,intent,label
0,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,주문,0.0
1,실수로 취소하면 재주문해야 하는 거죠?,주문,0.0
2,택배비 따로 추가되나요?,배송,1.0
3,택배비 있나요?,배송,1.0
4,택배비 따로 들어요,배송,1.0
...,...,...,...
79612,맨투맨 3가지 사이즈로만 나왔어요?,제품,4.0
79613,사이즈가 80A 인데 윗가슴 부분이 들뜨지 않나요?,제품,4.0
79614,사이즈가 29 - 30 정도면 라지인가요?,제품,4.0
79615,혹시나 M사이즈는 없죠?,제품,4.0


In [4]:
df['id'] = range(1, len(df) + 1)

In [5]:
import re

# 주어진 규칙에 따라 태그를 결정하는 함수를 정의한다
def tag_word(sentence_id, word, label):
    """
    문장 ID, 단어, 라벨을 입력으로 받아서 해당 단어에 부여할 태그를 결정한다.
    """
    # 1-1. XX시, YY분 형식일 경우 B-TI를 부여한다
    if re.search(r'\d+시|\d+분', word):
        return 'B-TI'
    
    time = ['아침', '점심', '저녁', '밤', '새벽', '자정']
    if word in time:
        return 'B-TI'
    
    # 1-2. 레오롱스커트, 맨투맨, 청바지는 B-PRODUCT를 부여한다
    if word in ['레오롱스커트', '맨투맨', '청바지']:
        return 'B-PRODUCT'
    
    # 1-3. 지역명에 대해 B-LOCATION을 부여한다
    locations = ['서울', '인천', '경기', '강원', '충북', '충남', '충청', '세종', 
                 '대전', '전남', '전북', '광주', '전라', '경북', '경남', '경상', 
                 '부산', '울산', '대구']
    if any(loc in word for loc in locations):
        return 'B-LOCATION'
    
    # 2, 3. 4번 라벨과 5번 라벨에 대한 규칙
    if label == 4.0 or label == 5.0:
        if re.search(r'\d+[A-Za-z]+|\d+사이즈|\d+', word):
            return 'B-SIZE'
    
    # 1-4. 위의 규칙에 해당하지 않는 단어는 'O'를 부여한다
    return 'O'

# 각 문장을 단어 단위로 분리하고, 각 단어에 대한 태그를 결정하는 함수를 정의한다
def process_sentence(sentence_id, sentence, label):
    """
    문장 ID, 문장, 라벨을 입력으로 받아서 해당 문장을 단어 단위로 분리하고,
    각 단어에 부여할 태그를 결정하여 '문장ID, 단어, 개체' 형태로 데이터를 반환한다.
    """
    # 문장을 단어 단위로 분리한다
    words = sentence.split()
    
    # 각 단어에 대한 태그를 결정한다
    tagged_words = [(sentence_id, word, tag_word(sentence_id, word, label)) for word in words]
    
    return tagged_words

# 테스트
sentence_id = 1
sentence = "아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?"
label = 0.0
process_sentence(sentence_id, sentence, label)


[(1, '아침도', 'O'),
 (1, '아니고', 'O'),
 (1, '밤', 'B-TI'),
 (1, '12시', 'B-TI'),
 (1, '30분에', 'B-TI'),
 (1, '결제한', 'O'),
 (1, '건데도', 'O'),
 (1, '그런가요?', 'O')]

In [6]:
df


,query,intent,label,id
0,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,주문,0.0,1
1,실수로 취소하면 재주문해야 하는 거죠?,주문,0.0,2
2,택배비 따로 추가되나요?,배송,1.0,3
3,택배비 있나요?,배송,1.0,4
4,택배비 따로 들어요,배송,1.0,5
...,...,...,...,...
79612,맨투맨 3가지 사이즈로만 나왔어요?,제품,4.0,79613
79613,사이즈가 80A 인데 윗가슴 부분이 들뜨지 않나요?,제품,4.0,79614
79614,사이즈가 29 - 30 정도면 라지인가요?,제품,4.0,79615
79615,혹시나 M사이즈는 없죠?,제품,4.0,79616


In [11]:
from ai.myapp.chatbot.utils.Preprocess import *

p = Preprocess(userdic='/home/azureuser/projects/aerius/ai/myapp/chatbot/dict/userdic.txt')

id_list=df['id'].tolist()
label_list=df['label'].tolist()
query_list=df['query'].tolist()

id_query_dict = dict(zip(id_list, query_list))

print(query_list[:5])

word_id_pairs = []
for k,v in id_query_dict.items():
    kk = k
    pp=p.delete_intent_trash_tags(v)
    word_list, _  = p.divide_words_tags(pp)

    word_id_pairs.extend((word, kk) for word in word_list)

word_id_dict = dict(word_id_pairs)
sample_result = {k: word_id_dict[k] for k in list(word_id_dict)[:5]}
sample_result



# new_df = []
# for sentence in query_list:
#     pp=p.delete_intent_trash_tags(sentence)
#     new_df.append(pp)

# new_df[:5]

# words = []
# for preprocessed in new_df:
#     word_list, _ = p.divide_words_tags(preprocessed)
#     words.extend(word_list)

# words[:5]


/home/azureuser/projects/aerius/ai/myapp/chatbot/dict/userdic.txt
['아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?', '실수로 취소하면 재주문해야 하는 거죠?', '택배비 따로 추가되나요?', '택배비 있나요?', '택배비 따로 들어요']


{'아침': 72479, '아니': 79585, '고': 79602, '밤': 78262, '12시': 60350}

In [10]:
values_list[:5]

['아침', '아니', '고', '밤', '12시']